imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import requests

helper function to format response objects into pandas dataframes

In [2]:
def call_endpoint(url, max_level=3, include_new_player_attributes=False):
    '''
    takes: 
        - url (str): the API endpoint to call
        - max_level (int): level of json normalizing to apply
        - include_player_attributes (bool): whether to include player object attributes in the returned dataframe
    returns:
        - df (pd.DataFrame): a dataframe of the call response content
    '''
    resp = requests.get(url).json()
    data = pd.json_normalize(resp['data'], max_level=max_level)
    included = pd.json_normalize(resp['included'], max_level=max_level)
    if include_new_player_attributes:
        inc_cop = included[included['type'] == 'new_player'].copy().dropna(axis=1)
        data = pd.merge(data
                        , inc_cop
                        , how='left'
                        , left_on=['relationships.new_player.data.id'
                                   ,'relationships.new_player.data.type']
                        , right_on=['id', 'type']
                        , suffixes=('', '_new_player'))
    return data

calling leagues endpoint

In [3]:
url = 'http://partner-api.prizepicks.com/leagues' # leagues that are live on our board
df = call_endpoint(url)
df.head(3)

,id,type,attributes.name,attributes.rank,attributes.active,attributes.created_at,attributes.updated_at,attributes.icon,attributes.image_url,attributes.projections_count,relationships.projection_filters.data
0,9,league,NFL,0,True,2018-07-19T13:26:11.622-04:00,2021-09-12T02:26:05.481-04:00,football,https://static.prizepicks.com/images/leagues/f...,458,"[{'id': '1', 'type': 'projection_filter'}, {'i..."
1,25,league,NFL2H,1,True,2019-02-03T20:00:47.421-05:00,2021-09-12T02:26:05.486-04:00,football,https://static.prizepicks.com/images/leagues/f...,0,[]
2,150,league,CFB2H,2,True,2020-09-23T10:34:52.470-04:00,2021-09-12T00:41:48.343-04:00,football,https://static.prizepicks.com/images/leagues/f...,0,[]


calling projections endpoint

In [4]:
url = 'https://partner-api.prizepicks.com/projections?per_page=1000' # use per_page=1000 to avoid pagination issues
df = call_endpoint(url)
df.head(3)

,id,type,attributes.line_score,attributes.risk_exposure,attributes.created_at,attributes.updated_at,attributes.description,attributes.start_time,attributes.status,attributes.board_time,attributes.rank,attributes.is_promo,attributes.discount_percentage,attributes.end_time,attributes.tv_channel,attributes.starting_pitcher,attributes.stat_type,attributes.projection_type,relationships.league.data.id,relationships.league.data.type,relationships.position.data,relationships.duration.data,relationships.stat_type.data.id,relationships.stat_type.data.type,relationships.new_player.data.id,relationships.new_player.data.type,relationships.position.data.id,relationships.position.data.type,relationships.duration.data.id,relationships.duration.data.type
0,253990,projection,8.0,None,2021-09-11T16:43:17.703-04:00,2021-09-11T16:51:18.412-04:00,DET,2021-09-12T13:00:00.000-04:00,pre_game,2021-08-26T00:00:00.000-04:00,1,False,None,None,None,None,Fantasy Score,Fantasy Score,46,league,NaN,NaN,14,stat_type,6701,new_player,NaN,NaN,NaN,NaN
1,170117,projection,400.0,None,2021-05-11T10:06:43.911-04:00,2021-08-11T11:23:12.391-04:00,2021-22 Season,2021-09-12T13:00:00.000-04:00,pre_game,2021-05-01T10:00:00.000-04:00,1,False,None,None,None,None,Fantasy Score,Fantasy Score,163,league,NaN,NaN,14,stat_type,39180,new_player,NaN,NaN,NaN,NaN
2,253771,projection,57.0,None,2021-09-11T10:53:14.667-04:00,2021-09-12T01:42:54.380-04:00,Round 4 (Par 63),2021-09-12T14:50:00.000-04:00,pre_game,2021-09-11T16:00:00.000-04:00,1,False,None,None,None,None,Fantasy Score,Fantasy Score,157,league,NaN,NaN,14,stat_type,28506,new_player,NaN,NaN,NaN,NaN


calling projections endpoint with filters

In [5]:
url = 'https://partner-api.prizepicks.com/projections?league_id=9&per_page=10'
df = call_endpoint(url)
df.head(3)

,id,type,attributes.line_score,attributes.risk_exposure,attributes.created_at,attributes.updated_at,attributes.description,attributes.start_time,attributes.status,attributes.board_time,attributes.rank,attributes.is_promo,attributes.discount_percentage,attributes.end_time,attributes.tv_channel,attributes.starting_pitcher,attributes.stat_type,attributes.projection_type,relationships.league.data.id,relationships.league.data.type,relationships.position.data.id,relationships.position.data.type,relationships.duration.data,relationships.stat_type.data.id,relationships.stat_type.data.type,relationships.new_player.data.id,relationships.new_player.data.type
0,238931,projection,0.5,None,2021-08-23T17:25:49.983-04:00,2021-09-11T12:45:40.142-04:00,CLE,2021-09-12T16:25:00.000-04:00,pre_game,2021-01-02T13:05:00.000-05:00,1,True,0.95,2021-09-12T16:45:38.278-04:00,None,None,Pass Yards,Single Stat,9,league,3,position,None,16,stat_type,1093,new_player
1,252599,projection,19.5,None,2021-09-10T10:13:21.833-04:00,2021-09-11T11:19:36.666-04:00,ATL,2021-09-12T13:00:00.000-04:00,pre_game,2021-08-26T00:00:00.000-04:00,21,False,NaN,None,None,None,Pass Completions,Single Stat,9,league,3,position,None,27,stat_type,24941,new_player
2,251415,projection,239.5,None,2021-09-08T13:46:41.610-04:00,2021-09-11T11:19:36.648-04:00,ATL,2021-09-12T13:00:00.000-04:00,pre_game,2021-04-01T19:05:00.000-04:00,21,False,NaN,None,None,None,Pass Yards,Single Stat,9,league,3,position,None,16,stat_type,24941,new_player


calling projections endpoint with filters and including new_player level data (like name and team)

In [6]:
url = 'https://partner-api.prizepicks.com/projections?league_id=9&per_page=1000'
df = call_endpoint(url, include_new_player_attributes=True)
df.head(3)

,id,type,attributes.line_score,attributes.risk_exposure,attributes.created_at,attributes.updated_at,attributes.description,attributes.start_time,attributes.status,attributes.board_time,attributes.rank,attributes.is_promo,attributes.discount_percentage,attributes.end_time,attributes.tv_channel,attributes.starting_pitcher,attributes.stat_type,attributes.projection_type,relationships.league.data.id,relationships.league.data.type,relationships.position.data.id,relationships.position.data.type,relationships.duration.data,relationships.stat_type.data.id,relationships.stat_type.data.type,relationships.new_player.data.id,relationships.new_player.data.type,relationships.position.data,id_new_player,type_new_player,attributes.name,attributes.created_at_new_player,attributes.updated_at_new_player,attributes.image_url,attributes.position,attributes.team,attributes.league_id,attributes.league,relationships.league.data.id_new_player,relationships.league.data.type_new_player
0,238931,projection,0.5,None,2021-08-23T17:25:49.983-04:00,2021-09-11T12:45:40.142-04:00,CLE,2021-09-12T16:25:00.000-04:00,pre_game,2021-01-02T13:05:00.000-05:00,1,True,0.95,2021-09-12T16:45:38.278-04:00,None,None,Pass Yards,Single Stat,9,league,3,position,None,16,stat_type,1093,new_player,NaN,1093,new_player,Patrick Mahomes,2018-09-20T14:27:59.218-04:00,2020-07-30T18:28:24.765-04:00,https://s3-us-west-2.amazonaws.com/static.fant...,QB,KC,9.0,NFL,9,league
1,251417,projection,50.5,None,2021-09-08T13:46:41.807-04:00,2021-09-11T11:19:36.658-04:00,ATL,2021-09-12T13:00:00.000-04:00,pre_game,2021-04-01T19:05:00.000-04:00,21,False,NaN,None,None,None,Rush Yards,Single Stat,9,league,3,position,None,15,stat_type,24941,new_player,NaN,24941,new_player,Jalen Hurts,2020-12-08T07:22:24.176-05:00,2020-12-08T07:22:24.176-05:00,https://s3-us-west-2.amazonaws.com/static.fant...,QB,PHI,9.0,NFL,9,league
2,251415,projection,239.5,None,2021-09-08T13:46:41.610-04:00,2021-09-11T11:19:36.648-04:00,ATL,2021-09-12T13:00:00.000-04:00,pre_game,2021-04-01T19:05:00.000-04:00,21,False,NaN,None,None,None,Pass Yards,Single Stat,9,league,3,position,None,16,stat_type,24941,new_player,NaN,24941,new_player,Jalen Hurts,2020-12-08T07:22:24.176-05:00,2020-12-08T07:22:24.176-05:00,https://s3-us-west-2.amazonaws.com/static.fant...,QB,PHI,9.0,NFL,9,league
